# Phrase Encoding Model Predictions

**Authorship:**
Adam Klie, *04/18/2022*
***
**Description:**
Notebook to do some cool stuff
***
<div class="alert alert-block alert-warning">
<b>TODOs</b>:
<ul>
    <b><li>Load the simplest dataset in</li></b>
    <b><li>Train a predictive model as baseline</li></b>
    <b><li></li></b>
    </ul>
</div>

# Set-up

In [2]:
# Classics
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pickle

# Figure style
plt.style.use('presentation')

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

import sys
sys.path.append('/cellar/users/aklie/projects/EUGENE/bin/')
import project_utils

In [3]:
# Define parameters of the notebook
DATASET = "2021_OLS_Library"  # Which dataset to look at
FEATURES = "phrase_encodings"  # What features to use to train the model

# Define training
SPLIT = 0.9
SUBSET = None
HYPERPARAM = "baseline"
MODEL = "LR"

# Fit models for multiple encodings

In [289]:
files = np.array(glob.glob("../data/{}/{}/*False.tsv".format(DATASET, FEATURES)))
files = files[np.argsort([len(file) for file in files])]

In [ ]:
files = glob.glob("../data/{}/{}/*.tsv".format(DATASET, FEATURES))
for i, file in enumerate(files):
    if os.path.getsize(file) <= 1e9:
        FILENAME = file.split("/")[-1].replace(".tsv", "")
        print(FILENAME)
        OUTDIR="../results/phrase/{}/{}".format(MODEL, FILENAME)
        if not os.path.exists(OUTDIR):
            os.makedirs(OUTDIR)
    
        data = pd.read_csv("../data/{}/{}/{}.tsv".format(DATASET, FEATURES, FILENAME), sep="\t", low_memory=False)
        x_cols = data.columns[data.columns.str.contains("((e.*|E.*|g.*|G.*)_)+")]

        # Use Joe's labels
        data["LABEL"] = np.nan
        data["LABEL"] = np.where(data["NONFUNC-<0.625"]=="True", 0, np.nan)
        data["LABEL"] = np.where(data["FUNC->0.685"]=="True", 1, data["LABEL"])

        if 'ACT_SumRNA_LenDNA_LOG2_NORM' in x_cols:
            x_cols = x_cols.drop('ACT_SumRNA_LenDNA_LOG2_NORM')
        
        training_data = data[~data["LABEL"].isna()]
        X = training_data[x_cols].values
        y = training_data["LABEL"].values
        
        # Split em
        X_train, X_test, y_train, y_test = split_train_test(X, y, split=SPLIT, subset=SUBSET, rand_state=13, shuf=True)

        # Standardize em
        print("Standardizing features")
        X_train, X_test = standardize_features(X_train, X_test)
        
        if MODEL == "LR":
            clf = LogisticRegression(random_state=13, verbose=1)
            print("Training a new LR model with the following params: {}".format(clf.get_params()))
        elif MODEL == "RF":
            clf = RandomForestClassifier(random_state=13, verbose=1)
            print("Training a new RF model with the following params: {}".format(clf.get_params()))
        clf.fit(X_train, y_train)
        
        file = "../results/phrase/{0}/{1}/params.csv".format(MODEL, FILENAME)
        print("Saving hyperparams to {}".format(file))
        pd.DataFrame(pd.Series(clf.get_params())).T.to_csv(file, index=False)
        
        # Get the classification results on validation set
        prob_thresh = 0.5
        y_tr_probs = clf.predict_proba(X_train)[:, 1]
        y_probs = clf.predict_proba(X_test)[:, 1]
        y_tr_preds = (y_tr_probs >= prob_thresh).astype(int)
        y_preds = (y_probs >= prob_thresh).astype(int)

        # Generate a report
        project_utils.classification_report(out_path="../results/phrase/{0}/{1}".format(MODEL, FILENAME),
                                            train_X=X_train, test_X=X_test, 
                                            train_y=y_train, test_y=y_test,
                                            train_preds=y_tr_preds, test_preds=y_preds,
                                            train_probs=y_tr_probs, test_probs=y_probs)
        #project_utils.coefficient_plot(clf, x_cols, top=20, title="{} Encoded Coefficients".format(FILENAME))
        file = "../results/phrase/{}/{}/clf_{}.pickle".format(MODEL, FILENAME, HYPERPARAM)
        pickle.dump(clf, open(file, 'wb'))
    #if i == 3:
    #    break

parse_ord_ori_aff_2_weightedByCount_True


/cellar/users/aklie/opt/miniconda3/envs/ml_env/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


Standardizing features
Training a new LR model with the following params: {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 13, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 1, 'warm_start': False}


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s finished


Saving hyperparams to ../results/phrase/LR/parse_ord_ori_aff_2_weightedByCount_True/params.csv
Predictions provided, skipping them
Generating confusion matrix
Calculating classification metrics
Metric	Train	Test
Accuracy	0.6604	0.6563
Precision	0.5617	0.5622
Recall	0.0949	0.0955
F0.1-Score	0.5357	0.5362
F0.5-Score	0.2832	0.2842
F1-Score	0.1624	0.1632
F2-Score	0.1139	0.1145
F10-Score	0.0957	0.0963
Plotting PR Curve
Plotting ROC Curve
Generating report
parse_ord_ori_spc_2_weightedByCount_True


...

Accuracy	0.6531	0.6489
Precision	0.0000	0.0000
Recall	0.0000	0.0000
F0.1-Score	nan	nan
F0.5-Score	nan	nan
F1-Score	nan	nan
F2-Score	nan	nan
F10-Score	nan	nan
Plotting PR Curve
Plotting ROC Curve
Generating report
parse_ord_ori_3_weightedByCount_False


/cellar/users/aklie/opt/miniconda3/envs/ml_env/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


Standardizing features
Training a new LR model with the following params: {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 13, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 1, 'warm_start': False}


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.3s finished


Saving hyperparams to ../results/phrase/LR/parse_ord_ori_3_weightedByCount_False/params.csv
Predictions provided, skipping them
Generating confusion matrix
Calculating classification metrics
Metric	Train	Test
Accuracy	0.6580	0.6561
Precision	0.5542	0.5769
Recall	0.0726	0.0772
F0.1-Score	0.5200	0.5422
F0.5-Score	0.2383	0.2514
F1-Score	0.1284	0.1362
F2-Score	0.0879	0.0934
F10-Score	0.0733	0.0779
Plotting PR Curve
Plotting ROC Curve


# Load data

In [4]:
# Define data details
PHRASE = "ord"
COUNTS = False
LEN = 2
FILENAME = "parse_{0}_{1}_weightedByCount_{2}".format(PHRASE, LEN, COUNTS)
OUTDIR="../results/phrase/{}/{}".format(MODEL, FILENAME)
if not os.path.exists(OUTDIR):
    os.makedirs(OUTDIR)

In [5]:
data = pd.read_csv("../data/{}/{}/{}.tsv".format(DATASET, FEATURES, FILENAME), sep="\t", low_memory=False)
x_cols = data.columns[data.columns.str.contains("((e.*|E.*|g.*|G.*)_)+")]

# Use Joe's labels
data["LABEL"] = np.nan
data["LABEL"] = np.where(data["NONFUNC-<0.625"]=="True", 0, np.nan)
data["LABEL"] = np.where(data["FUNC->0.685"]=="True", 1, data["LABEL"])

# Use my own
#data["LABEL"] = np.nan
#data['ACT_SumRNA_LenDNA_LOG2_NORM'] = data['ACT_SumRNA_LenDNA_LOG2_NORM'].replace({"ND":np.nan})
#data["ACT_SumRNA_LenDNA_LOG2_NORM"] = data["ACT_SumRNA_LenDNA_LOG2_NORM"].astype(float)
#data.loc[data['ACT_SumRNA_LenDNA_LOG2_NORM'] <= 0.585, "LABEL"] = 0
#data.loc[data['ACT_SumRNA_LenDNA_LOG2_NORM'] >= 0.685, "LABEL"] = 1

if 'ACT_SumRNA_LenDNA_LOG2_NORM' in x_cols:
    x_cols = x_cols.drop('ACT_SumRNA_LenDNA_LOG2_NORM')
data["LABEL"].value_counts(), x_cols

/cellar/users/aklie/opt/miniconda3/envs/ml_env/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


(0.0    177023
 1.0     94194
 Name: LABEL, dtype: int64,
 Index(['E_G', 'G_G', 'E_E', 'G_E'], dtype='object'))

In [6]:
training_data = data[~data["LABEL"].isna()]
X = training_data[x_cols]
y = training_data["LABEL"]
X.shape, y.shape

((271217, 4), (271217,))

In [7]:
from project_utils import split_train_test, standardize_features

# Split em
X_train, X_test, y_train, y_test = split_train_test(X, y, split=SPLIT, subset=SUBSET, rand_state=13, shuf=True)

# Check em
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((244095, 4), (244095,), (27122, 4), (27122,))

In [8]:
X_train.mean(axis=0), X_train.std(axis=0), X_test.mean(axis=0), X_test.std(axis=0)

(E_G    0.900453
 G_G    1.000000
 E_E    1.000000
 G_E    0.899129
 dtype: float64,
 E_G    0.299396
 G_G    0.000000
 E_E    0.000000
 G_E    0.301158
 dtype: float64,
 E_G    0.902146
 G_G    1.000000
 E_E    1.000000
 G_E    0.896542
 dtype: float64,
 E_G    0.297123
 G_G    0.000000
 E_E    0.000000
 G_E    0.304562
 dtype: float64)

In [14]:
np.unique(X, axis=0, return_counts=True)

(array([[0, 1, 1, 1],
        [1, 1, 1, 0],
        [1, 1, 1, 1]]),
 array([ 26953,  27428, 216836]))

# Train a model

In [10]:
import statsmodels.api as sm

In [11]:
log_reg = sm.Logit(y_train, sm.tools.add_constant(X_train, has_constant="add"))

In [20]:
res = log_reg.fit(method="nm")

/cellar/users/aklie/opt/miniconda3/envs/ml_env/lib/python3.8/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/cellar/users/aklie/opt/miniconda3/envs/ml_env/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [21]:
res.pvalues

const   NaN
E_G     NaN
G_G     NaN
E_E     NaN
G_E     NaN
dtype: float64

In [263]:
res.predict(sm.tools.add_constant(X_test, has_constant="add"))

array([0.30430767, 0.3583055 , 0.3583055 , ..., 0.3583055 , 0.3583055 ,
       0.3583055 ])

In [264]:
# Get the classification results on validation set
prob_thresh = 0.5
y_tr_probs = res.predict(sm.tools.add_constant(X_train, has_constant="add"))
y_probs = res.predict(sm.tools.add_constant(X_test, has_constant="add"))
y_tr_preds = (y_tr_probs >= prob_thresh).astype(int)
y_preds = (y_probs >= prob_thresh).astype(int)

In [265]:
# Generate a report
project_utils.classification_report(out_path="../results/phrase/{0}/{1}".format(MODEL, FILENAME),
                                    train_X=X_train, test_X=X_test, 
                                    train_y=y_train, test_y=y_test,
                                    train_preds=y_tr_preds, test_preds=y_preds,
                                    train_probs=y_tr_probs, test_probs=y_probs)

Predictions provided, skipping them
Generating confusion matrix
Calculating classification metrics
Metric	Train	Test


/cellar/users/aklie/opt/miniconda3/envs/ml_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/cellar/users/aklie/opt/miniconda3/envs/ml_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/cellar/users/aklie/projects/EUGENE/bin/project_utils.py:282: RuntimeWarning: invalid value encountered in double_scalars
  return (1+(beta**2))*((pr*rec)/(((beta**2)*pr)+rec))


Accuracy	0.6531	0.6489
Precision	0.0000	0.0000
Recall	0.0000	0.0000
F0.1-Score	nan	nan
F0.5-Score	nan	nan
F1-Score	nan	nan
F2-Score	nan	nan
F10-Score	nan	nan
Plotting PR Curve
Plotting ROC Curve
Generating report


In [144]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

#### <u> **Train new model from scratch using out-of-box or hand-crafted hyperparams** </u> 

In [146]:
if TRAIN == True and HYPERPARAM == "baseline":
    if MODEL == "LR":
        clf = LogisticRegression(random_state=13, verbose=1)
        print("Training a new LR model with the following params: {}".format(clf.get_params()))
    elif MODEL == "RF":
        clf = RandomForestClassifier(random_state=13, verbose=1)
        print("Training a new RF model with the following params: {}".format(clf.get_params()))
    clf.fit(X_train, y_train)

Training a new RF model with the following params: {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 13, 'verbose': 1, 'warm_start': False}


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.8s finished


#### <u> **Save hyperpameters of model** </u>

In [147]:
if TRAIN == True:
    file = "../results/phrase/{0}/{1}/params.csv".format(MODEL, FILENAME)
    print("Saving hyperparams to {}".format(file))
    pd.DataFrame(pd.Series(clf.get_params())).T.to_csv(file, index=False)
else:
    print("Not training a new model, load in cell below")

Saving hyperparams to ../results/phrase/RF/parse_ord_2_weightedByCount_False/params.csv


# Validation Set Performance

In [148]:
# Get the classification results on validation set
prob_thresh = 0.5
y_tr_probs = clf.predict_proba(X_train)[:, 1]
y_probs = clf.predict_proba(X_test)[:, 1]
y_tr_preds = (y_tr_probs >= prob_thresh).astype(int)
y_preds = (y_probs >= prob_thresh).astype(int)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


In [149]:
# Generate a report
project_utils.classification_report(out_path="../results/phrase/{0}/{1}".format(MODEL, FILENAME),
                                    train_X=X_train, test_X=X_test, 
                                    train_y=y_train, test_y=y_test,
                                    train_preds=y_tr_preds, test_preds=y_preds,
                                    train_probs=y_tr_probs, test_probs=y_probs)

Predictions provided, skipping them
Generating confusion matrix
Calculating classification metrics
Metric	Train	Test
Accuracy	0.5280	0.5229
Precision	0.5245	0.5186
Recall	0.8261	0.8248
F0.1-Score	0.5264	0.5205
F0.5-Score	0.5658	0.5602
F1-Score	0.6416	0.6368
F2-Score	0.7409	0.7377
F10-Score	0.8214	0.8200
Plotting PR Curve
Plotting ROC Curve
Generating report


In [150]:
project_utils.coefficient_plot(clf, x_cols, top=20, title="{} Encoded Coefficients".format(FILENAME))

AttributeError: 'RandomForestClassifier' object has no attribute 'coef_'

# Save the classifier

In [126]:
if TRAIN == True:
    file = "../results/phrase/{}/{}/clf_{}.pickle".format(MODEL, FILENAME, HYPERPARAM)
    pickle.dump(clf, open(file, 'wb'))
else:
    print("No need to save, loaded trained model already")

# References